In [ ]:
import pandas as pd
import numpy as np
import random
from math import ceil, sqrt
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

def svgCloudEdge(x0, y0, x1, y1, h=20, r=14, rd=10, _stroke_='#ff0000', _fill_='#ff0000'):    
    vx, vy   = x1 - x0, y1 - y0
    l        = sqrt(vx * vx + vy * vy)
    vx, vy   = vx / l, vy / l
    vxp, vyp = vy, -vx
    _svg_ = f'<path d="m {x0} {y0} l {vxp*h} {vyp*h} l {vx*l} {vy*l} l {-vxp*h} {-vyp*h} z" stroke="{_stroke_}" fill="{_fill_}" />'

    circles = ceil(l / r)
    for i in range(int(circles)+1):
        x, y = x0 + vxp*h + vx*l*i/circles, y0 + vyp*h + vy*l*i/circles
        rr, xr, yr = r + rd*(random.random()-0.5), x + rd*(random.random()-0.5), y + rd*(random.random()-0.5)
        _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'

    rr     = h/2
    xr, yr = x0 + vxp*rr, y0 + vyp*rr
    _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'

    rr     = h/2
    xr, yr = x0 + vxp*rr + l*vx, y0 + vyp*rr + l*vy
    _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'
    return _svg_


In [ ]:
_svg_ =  '<svg width="300" height="300">'
_svg_ += '<rect x="0" y="0" width="300" height="300" fill="#ffffff"/>'
_svg_ += svgCloudEdge(100,  50, 200,  50)
_svg_ += svgCloudEdge(200,  50, 200, 150)
_svg_ += svgCloudEdge(200, 150, 100, 150)
_svg_ += svgCloudEdge(100, 150, 100,  50)
_svg_ += '</svg>'
rt.displaySVG(_svg_)

In [ ]:
r

In [ ]:
rd